In [94]:
from keras.datasets import mnist 
import numpy as np
from sklearn.metrics import accuracy_score

#*Importing the dataset

In [95]:
(train_images , train_labels),(test_images , test_labels) = mnist.load_data()
test_labels.shape

(10000,)

#*subset the data to use only class 0 and class1

In [96]:
train_indexes = (train_labels==0)|(train_labels==1)
test_indexes = (test_labels==0)|(test_labels==1)
x_train = train_images[train_indexes]
y_train = train_labels[train_indexes]

x_test = test_images[test_indexes]
y_test = test_labels[test_indexes]

#reshape training data
x_train = x_train.reshape(-1 , 784)
y_train = y_train.reshape(-1 , 1)

x_test = x_test.reshape(-1 , 784)
y_test = y_test.reshape(-1 , 1)

In [97]:
y_test.shape

(2115, 1)

#*standardization of the data

In [98]:
mean_x_train = np.mean(x_train)
std_x_train = np.std(x_train)
x_train = (x_train - mean_x_train)/std_x_train


x_test = (x_test -mean_x_train)/std_x_train

In [99]:
y_test.shape

(2115, 1)

#split the data imto validation data and training data

In [100]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

#* implementing Logistic Regression

In [101]:
print(x_train.shape)
print(y_train.shape)
print(y_val.shape)
print(x_train.shape[0])

(10132, 784)
(10132, 1)
(2533, 1)
10132


In [102]:
def logistic_regression_L1(X, y, lr, num_iterations , lambda_param):
        
        m , n = X.shape     # m is the number of sampes  and n is the number  of features 
        w = np.zeros((n , 1))
        b = 0

        for i in range(num_iterations):
            z = np.dot(X, w) + b
            A = 1 / (1 + np.exp(-z))

            # calculate regularized cost function
            J = -1/n * np.sum(y*np.log(A) + (1-y)*np.log(1-A)) + lambda_param * np.sum(np.abs(w))


             # calculate partial derivatives
            dw = 1/m * np.dot(X.T, (A-y)) + lambda_param * np.sign(w)
            db = 1/m * np.sum(A-y)

            # update weights and bias
            w = w - lr * dw
            b = b - lr * db

            return w , b


In [103]:
w ,b = logistic_regression_L1(x_train , y_train ,0.01 , 1000 , 0.01)       #L1 when lambda = 0.01

z = np.dot(x_test, w) + b
A = 1 / (1 + np.exp(-z))
y_pred = (A > 0.5).astype(int) 
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2%}".format(acc))


w ,b = logistic_regression_L1(x_train , y_train ,0.01 , 1000 ,0)         #L1 when lambda = 0.1
z = np.dot(x_test, w) + b
A = 1 / (1 + np.exp(-z))
y_pred2 = (A > 0.5).astype(int)
acc = accuracy_score(y_test, y_pred2)
print("Accuracy2: {:.2%}".format(acc))


Accuracy: 99.67%
Accuracy2: 99.67%


#Logistic Regression with mini-batch gradient descent

In [104]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Define the mini-batch gradient descent function 
def mini_batch_gradient_descent(x , y , batch_size , ):
    num_iterations = 1000
    lr = 0.001
    w = np.zeros((x.shape[1], 1))
    b = 0
    N = len(y)
    num_batches = int(np.ceil(N / batch_size))
    losses = []
    for i in range(num_iterations):
        for i in range(num_batches):
            batch_start = i * batch_size
            batch_end = min(batch_start + batch_size, N)
            x_batch = x[batch_start:batch_end]
            y_batch = y[batch_start:batch_end]
            y_hat = np.dot(x_batch, w) + b
            dw = (2/batch_size) * np.dot(x_batch.T, (y_hat - y_batch))
            db = (2/batch_size) * np.sum(y_hat - y_batch)
            w = w - lr * dw
            b = b - lr * db
            
    return w, b


In [111]:
batch_sizes = [128, 256]
num_iterations = 1000
for batch_size in batch_sizes:
    w,b = mini_batch_gradient_descent(x_train, y_train ,batch_size)
    mini_batch_acc = np.mean((sigmoid(np.dot(x_test, w)+b) >= 0.5) == y_test)
    print(f"batch_size: {batch_size:.2f},  Accuracy: {mini_batch_acc:.4f}")

batch_size: 128.00,  Accuracy: 0.7641
batch_size: 256.00,  Accuracy: 0.7546


#RMS-prop Optimizer

In [106]:
import math
def Rmsprop_optimizer(x ,y,beta,epsilon,iterations, alpha):
        m, n = x.shape
        w = np.zeros((n, 1))
        b = 0
        V_dw = np.zeros((n, 1))
        V_db = 0
        for i in range(iterations):
            z = np.dot(x,w)+b
            A = sigmoid(z)
            dw = np.mean(np.dot(2 * x.T, (A - y)))
            db = np.mean(A - y)
            #grad = 1/m * X.T * (h - y)
            V_dw = beta * V_dw + (1 - beta) * dw**2 / (1 - math.pow(beta,i+1))
            V_db = beta * V_db + (1 - beta) * db**2 / (1 - math.pow(beta,i+1))
            w = w - alpha / np.sqrt(V_dw + epsilon) * dw
            b = b - alpha / np.sqrt(V_db + epsilon) * db
        return w,b

In [107]:
w,b = Rmsprop_optimizer(x_train, y_train.reshape(-1, 1),beta=0.9,epsilon=1e-8,iterations=1000,alpha=0.01)

z=np.dot(x_test,w)+b
y_pred = (sigmoid(z) >= 0.5)
Rmsprop_accuracy = np.mean(y_pred == y_test.reshape(-1, 1))
print("Accuracy:{:.2%}".format(Rmsprop_accuracy))

Accuracy:93.19%


#Adam optimizer

In [108]:
def Adam_optimizer(x ,y,beta,epsilon, alpha , iterations):
        
        m, n = x.shape
        w = np.zeros((n, 1))
        b = 0
        V_dw = np.zeros((n, 1))
        V_db = 0
        momentum_w,momentum_b = 0,0
        update_w, update_b = 0,0
        beta_1 = 0.9
        beta_2 = 0.999
        for i in range(iterations):
            z = np.dot(x,w)+b
            A = sigmoid(z)
            dw = np.mean(np.dot(2 * x.T, (A - y)))
            db = np.mean(A - y)
            
            momentum_w = beta_1 * momentum_w + (1 - beta_1) * dw
            momentum_b = beta_1 * momentum_b + (1 - beta_1) * db
            update_w = beta_2 * update_w + (1 - beta_2) * np.square(dw)
            update_b = beta_2 * update_b + (1 - beta_2) * np.square(db)
            momentum_w = momentum_w / (1 - beta_1 ** i+1)
            momentum_b = momentum_b / (1 - beta_1 ** i+1)
            update_w = update_w / (1 - beta_2 ** i+1)
            update_b = update_b / (1 - beta_2 ** i+1)

            # Update weights and biases
            w -= alpha * momentum_w / (np.sqrt(update_w) + epsilon)
            b -= alpha * momentum_b / (np.sqrt(update_b) + epsilon)
        return w,b

In [109]:
w,b = Adam_optimizer(x_train, y_train.reshape(-1, 1),beta=0.9,epsilon=1e-8,iterations=1000,alpha=0.01)

z = np.dot(x_test , w) + b
y_pred = (sigmoid(z) >= 0.5).astype(int)
 
Adam_accuracy = np.mean(y_pred == y_test.reshape(-1, 1))
print("Accuracy:{:.2%}".format(Adam_accuracy)

Accuracy:93.33%


# reporting the accuracies


when we implement the L1 regulsrization with lambda= 0.01 we get accuracy = 99.67% (the highest accuracy we get in our case) and lambda =0.1 and get the same accuracy ,L1 gives high accuracy because it helps to prevent the overfitting .
*In mini_batch we use batch size 128 we get accuracy = 76.41% 
and batch size 256 get accuracy = 75.46%  wich might be because the data are relatively small and the two sizes numbers are close .
*For RMSprop and Adam optimizers , we obtain accuracy of 93.19% and 93.33%
Respectively Both optimizers have shown to be effective in deep learning and are widely used due to their ability to adapt the learning rate according to the gradients .